##### Some modules loaded, may/maynot be used further...  
Just loaded if required/felt neccessary

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
import os

from torchvision import transforms

import torch
import torch.nn as nn
from torchvision import models

import copy
"""import time

from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import random
from torchvision.transforms.functional import to_pil_image
"""


##### training dataframe loaded
- made it crisp
- gave unique encoded labels for it

In [23]:
# the list of directries being used : needed to change if required
train_df = pd.read_csv("/home/kaushik/Music/TL-ML/dataset/train.csv")
train_image_dir = "/home/kaushik/Music/TL-ML/dataset/train"

test_df = pd.read_csv("/home/kaushik/Music/TL-ML/dataset/test.csv")
test_image_dir = "/home/kaushik/Music/TL-ML/dataset/test"


In [24]:

#make the required changes to get labels and image names
train_df['image'] = train_df['Id'].astype(str)
train_df['label'] = train_df['Class'].str.strip()
train_df = train_df.drop(columns=['Id', 'Class'], axis=1)

#unique label encodes for each uni-classes(country+dominion here)
le = LabelEncoder()
train_df['label_encoded'] = le.fit_transform(train_df['label'])
#train_df.head(7)


##### train & validation split

In [25]:
#a map btw label numbers to unique classes (1-1 map)
label_to_class = dict(zip(le.transform(le.classes_), le.classes_))
# print(label_to_class)

#train-val split
train, val = train_test_split(train_df, test_size=0.20, stratify=train_df['label'], random_state=42)
print(len(train), len(val))


8294 2074


##### checking if path exits, 
- for an image(preferably- Id) .. if it's path exits, return the path,   
- else it returns path to `didn't exist` for 3 such image named `10211`, `11353`, `11270` . 
- these doesn't have any file type

In [ ]:
def check_if_exists(image_dir, image_id):
    img_path1 = os.path.join(image_dir, image_id + '.jpg')
    img_path2 = os.path.join(image_dir, image_id + '.png')
    img_path3 = os.path.join(image_dir, image_id + '.webp')
    if os.path.exists(img_path1):
        return img_path1
    elif os.path.exists(img_path2):
        return img_path2
    elif os.path.exists(img_path3):
        return img_path3
    else:
        print(f"[Missed] No image found for Id {image_id}")
        return "didn't exist"


##### the dataset of images  
- defined as a class

In [ ]:
class Dataset_for_coins(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.data = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        image_id = str(self.data.loc[idx, 'image'])
        img_path = check_if_exists(self.image_dir, image_id)
        
        if img_path == "didn't exist":  #assuming it's prev image belongs to the same category , so training it is same as training this one
            idx = idx -1
            image_id = str(self.data.loc[idx, 'image'])
            img_path = check_if_exists(self.image_dir, image_id)

        # Trying to load the image safely
        try:
            with Image.open(img_path) as img:
                image = img.convert("RGB")
                label = self.data.loc[idx, 'label_encoded']
        except Exception as e:
            # comming expicially for AVIF file types 
            print(f"[Unreadable - so skipping image] : {e}")
            # print(f"{img_path}")

        
        if self.transform:  #f transform was required at any stage
            try :
                image = self.transform(image)
            except UnboundLocalError: # got this error again dur to AVIF file types
                
                idx = idx -1
                image_id = str(self.data.loc[idx, 'image'])
                img_path = check_if_exists(self.image_dir, image_id)
                image = (Image.open(img_path)).convert("RGB")
                image = self.transform(image)
                label = self.data.loc[idx, 'label_encoded']
        return image, label


##### transformations for DataAugmentations here
- random rotations done @ 15
- scalling(for <224, it's not performing well & for >this it's taking more time .. thence kept it at 224) and flipping(horizontal)

In [28]:
# Resize images to fixed size (e.g., 224x224) for CNN input
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
])

val_and_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = Dataset_for_coins(train, train_image_dir, transform=train_transform)
val_dataset = Dataset_for_coins(val, train_image_dir, transform=val_and_test_transform)


##### loading images, pretrained ResNet18 Model also with adam's optimizer
- batch_size was kept @ 1 since i wanted to train individually  
- resnet18 i sloaded along with adam's optimizer

In [ ]:
# batch sizes were chusen empirically upon multiple guess test runs of the model
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=6, shuffle=False)

# print(len(train_loader))

# total number of classes
num_classes = len(label_to_class)

# Loading a pretrained ResNet18
model = models.resnet18(pretrained=True)

# Replacing the final fully connected layer
model.fc = nn.Linear(model.fc.in_features, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


/home/kaushik/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kaushik/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [30]:
# freezing all the layer and leaving of the final layer i.e. unfreeze only the final layer
for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True


##### a sanity check before starting
- gets the list of all the imge id's which doesn't have corresponding image in the directory

In [31]:
missing = []
for id_str in train_df['image']:
    found = any(os.path.exists(os.path.join(train_image_dir, id_str + ext)) for ext in ['.jpg', '.png', '.webp'])
    if not found:
        missing.append(id_str)

print(f"{len(missing)} missing images: {missing}")


3 missing images: ['10211', '11270', '11353']


##### training starts noww

In [32]:
# a trivial function
def compute_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    correct = (preds == labels).sum().item()
    acc = correct / len(labels)
    return acc

def training_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20, waiting=5):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    waiting_counter = 0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs} started")
        print("-" * 30)
        
        for phase in ['train', 'val']:
            # print("entered phase")
            if phase == 'train':
                model.train()
                dataloader = train_loader
            else:
                model.eval()
                dataloader = val_loader

            present_loss = 0.0
            present_acc = 0.0
            # print("about to entered for")
            
            for images, labels in dataloader:
                # print("entered for")
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    acc = compute_accuracy(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                present_loss += loss.item() * images.size(0)
                present_acc += acc * images.size(0)
                    
            epoch_loss = present_loss / len(dataloader.dataset)
            epoch_acc = 100* present_acc / len(dataloader.dataset)

            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.3f}%")
            print("-" * 30)
            print(" ")

            # saving model if validation improves and waiting for 5 epochs
            if phase == 'val':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    waiting_counter = 0
                else:
                    waiting_counter += 1
                    if waiting_counter >= waiting:
                        print("Early stopping now .")
                        model.load_state_dict(best_model_wts)
                        return model

    model.load_state_dict(best_model_wts)
    return model


In [ ]:
# advised to run the epochs for more than 10 epochs for better training : like 20-30
# to be noted that the training of the model(this particular cell) took arround 80 mins for 10 epochs
# (probably my machione didn't have a GPU and hence had to run on CPU)
model = training_model(
    model, 
    criterion, 
    optimizer, 
    train_loader, 
    val_loader, 
    num_epochs=10, 
    waiting=5
)


Epoch 1/10 started
------------------------------
[Unreadable - so skipping image] : cannot identify image file '/home/kaushik/Music/TL-ML/dataset/train/9041.jpg'
[Unreadable - so skipping image] : cannot identify image file '/home/kaushik/Music/TL-ML/dataset/train/9291.jpg'
[Unreadable - so skipping image] : cannot identify image file '/home/kaushik/Music/TL-ML/dataset/train/9320.jpg'
[Unreadable - so skipping image] : cannot identify image file '/home/kaushik/Music/TL-ML/dataset/train/8778.jpg'
[Missed] No image found for Id 11270
[Unreadable - so skipping image] : cannot identify image file '/home/kaushik/Music/TL-ML/dataset/train/9655.jpg'
[Unreadable - so skipping image] : cannot identify image file '/home/kaushik/Music/TL-ML/dataset/train/9061.jpg'
[Unreadable - so skipping image] : cannot identify image file '/home/kaushik/Music/TL-ML/dataset/train/9055.jpg'
[Unreadable - so skipping image] : cannot identify image file '/home/kaushik/Music/TL-ML/dataset/train/9493.jpg'
[Unreadab

##### testing noww
- outputing the `test.csv` as `test_output.csv` for distinction

In [ ]:
test_df['image'] = test_df['Id'].astype(str)
test_df['label'] = "placeholder"
test_df['label_encoded'] = -1
test_df = test_df.drop(columns=['Id'], axis=1)

# test_df.head()

test_dataset = Dataset_for_coins(test_df, test_image_dir, transform=val_and_test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

all_preds = []
for images, labels in test_loader:
    outputs = model(images)
    _, preds = torch.max(outputs, 1)
    all_preds.extend(preds.cpu().numpy())
    
# class_names = le.classes_
for i , pred in enumerate(all_preds):
    test_df.loc[i, 'label'] = label_to_class[pred]

test_df['Id'] = test_df['image'].astype(int)
test_df['Class'] = test_df['label'].str.strip()
test_df = test_df.drop(columns=['image', 'label', 'label_encoded'], axis=1)

test_df.to_csv("test_output.csv", index=False)


**A small report**  
1. since my pillow version didn't support `AVIF` file types, i had to train it in many ways, like  
    - removing files of these types during traning
    - training along with these files, 
    - finally modified the logic such that , if these can't be read, then it's previous image / `Id` is tried to train again .. legally covering for the missed training for that image.  

2. i have been facing errors (of various types) while doing the testing part like precision & F1-Scores and i am yet to figure it out why
3. the confusion matrix is very very clumsy with mathplotlib
    - i have been trying to find ways to depict / present the confusion matrix in a way such that we can be able to deduce some conclusions from the matrix image(since the original image of the matrix had arround $300 \times 300$ cells)

4. i have tried predicting the coin images from a few coin photos that i took @ home
    - ofcourse indian currancy with my not so good camera quality
    - i have observed a few deviations a couple of times probably due to the color of the coin + hue + saturation of the images taken , although it had done a good amount of predictions on 30 pictures taken by me of the indian coins kept @ my home


- stopping the progress right here